In [31]:
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import numpy as np
from sklearn.decomposition import PCA
import torch.nn.functional as F

In [32]:
# Define data augmentation
data_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomResizedCrop(256, scale=(0.8, 1.0)),
    transforms.ToTensor(),
])

In [33]:
# FlowerDataset class
class FlowerDataset(Dataset):
    def __init__(self, image_dir, labels_file, transform=None, pca_transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.pca_transform = pca_transform
        # Load image file names
        self.image_filenames = os.listdir(image_dir)
        print(f"Loaded {len(self.image_filenames)} images from {image_dir}")
        # Load labels from the labels file
        with open(labels_file, 'r') as f:
            # Subtract 1 from each label to make them zero-indexed
            self.labels = [int(line.strip()) - 1 for line in f.readlines()]  
        print(f"Loaded {len(self.labels)} labels from {labels_file}")

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_filenames[idx])
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        label = self.labels[idx]  # Labels should already be integers

        # Apply PCA transform if it exists
        if self.pca_transform is not None:
            flattened_image = image.view(-1).numpy()  # Flatten the image
            pca_image = self.pca_transform.transform(flattened_image.reshape(1, -1))  # Transform using PCA
            return torch.tensor(pca_image, dtype=torch.float32).squeeze(0), label  # Return PCA-transformed image

        return image, label  # Fallback to original image if PCA is not applied


In [34]:
# Paths to your image directory and labels file
image_dir = 'train_data'
labels_file = 'train_labels.txt'

# Create the dataset
train_dataset = FlowerDataset(image_dir=image_dir, labels_file=labels_file, transform=data_transforms)

# Collect flattened images for PCA
flattened_images = []
for image, _ in train_dataset:
    flattened_image = image.view(-1).numpy()  # Flatten and convert to numpy
    flattened_images.append(flattened_image)

# Apply PCAb
n_components = 100  # Adjust the number of components as needed
pca = PCA(n_components=n_components)
pca.fit(flattened_images)

# Now create the DataLoader with PCA transform
train_dataset = FlowerDataset(image_dir=image_dir, labels_file=labels_file, transform=data_transforms, pca_transform=pca)

# Create the DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Load validation dataset
val_image_dir = 'val_data'
val_labels_file = 'val_labels.txt'
val_dataset = FlowerDataset(image_dir=val_image_dir, labels_file=val_labels_file, transform=data_transforms, pca_transform=pca)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


Loaded 3000 images from train_data
Loaded 3000 labels from train_labels.txt
Loaded 3000 images from train_data
Loaded 3000 labels from train_labels.txt
Loaded 600 images from val_data
Loaded 600 labels from val_labels.txt


In [35]:
# Check number of images considered for training after augmentation
augmented_images_count = 0

# Loop through the DataLoader to count the total images generated
for data, _ in train_loader:
    augmented_images_count += data.size(0)  # Count the batch size

print(f'Total number of augmented images considered for training: {augmented_images_count}')


Total number of augmented images considered for training: 3000


In [18]:
# Define the FlowerNet model
class FlowerNet(nn.Module):
    def __init__(self, num_features, num_classes):  # Corrected method name
        super(FlowerNet, self).__init__()  # Corrected method name
        self.fc1 = nn.Linear(num_features, 512)  # Input layer
        self.fc2 = nn.Linear(512, 256)            # Hidden layer
        self.fc3 = nn.Linear(256, num_classes)    # Output layer
        self.dropout = nn.Dropout(0.3)             # Dropout layer for regularization
        self.norm = nn.LayerNorm(512)              # Layer normalization

    def forward(self, x):
        x = F.relu(self.fc1(x))                   # First layer with ReLU activation
        x = self.norm(x)                          # Normalization after activation
        x = F.relu(self.fc2(x))                   # Second layer with ReLU activation
        x = self.dropout(x)                       # Apply dropout
        x = self.fc3(x)                           # Output layer
        return x


In [11]:
# Set the number of features and classes
num_features = 100  # Adjust this based on the number of PCA components
num_classes = 60
model = FlowerNet(num_features, num_classes)

# Hyperparameters
learning_rate = 0.01
n_epochs = 30

# Set up criterion and optimizer
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [29]:
# Added this
def __getitem__(self, idx):
    img_path = os.path.join(self.image_dir, self.image_filenames[idx])
    image = Image.open(img_path).convert('RGB')
    
    if self.transform:
        image = self.transform(image)
    
    label = self.labels[idx]  # Labels should already be integers

    print(f"Label for image {idx}: {label}")  # Debugging line

    # Apply PCA transform if it exists
    if self.pca_transform is not None:
        flattened_image = image.view(-1).numpy()  # Flatten the image
        pca_image = self.pca_transform.transform(flattened_image.reshape(1, -1))  # Transform using PCA
        return torch.tensor(pca_image, dtype=torch.float32).squeeze(0), label  # Return PCA-transformed image

    return image, label  # Fallback to original image if PCA is not applied
print("Minimum label:", min(train_dataset.labels))
print("Maximum label:", max(train_dataset.labels))


Minimum label: 0
Maximum label: 59


In [25]:
#Added this
print(f"Label range in training dataset: min={min(train_dataset.labels)}, max={max(train_dataset.labels)}")
print(f'Training data size: {len(train_loader.dataset)}')
print(f'Sample from train loader: {next(iter(train_loader))}')


Label range in training dataset: min=0, max=59
Training data size: 3000
Sample from train loader: [tensor([[-62.1666, -51.9524,   0.7376,  ...,  -2.0148,   0.1644,  -2.1931],
        [-30.6468,  -2.0472,  82.2818,  ...,   3.1265,   2.2207,   3.1108],
        [ 64.6472, -36.2351,  16.2443,  ...,   2.1767,  -6.4202,  -1.6366],
        ...,
        [ 18.8139,  -1.4905,  -2.9047,  ...,   2.9370,  -0.3992,   0.5593],
        [  2.9876, -27.3472, -17.2954,  ...,  -0.7747,  -0.9865,   4.0756],
        [ 79.6175, -37.8105,  25.8011,  ...,   2.6600,   2.2295,   0.7990]]), tensor([17, 52, 52, 29, 13, 13, 28, 59, 16, 11, 52, 22,  2, 19, 32, 37, 40, 34,
        42, 30, 38, 45, 37, 39, 35, 24, 59,  7,  0, 19, 37, 11])]


In [27]:
# Make sure to define the device properly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move model to the selected device

# Training Loop
n_epochs = 1
for epoch in range(n_epochs):
    print(f"Starting Epoch {epoch + 1}/{n_epochs}")
    model.train()  # Set the model to training mode
    train_loss = 0.0
    correct = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        print(f"Processing batch {batch_idx + 1}/{len(train_loader)}")  # Debugging print
        # Move data and target to the same device as the model
        data = data.to(device)  
        target = target.to(device)

        optimizer.zero_grad()  # Clear gradients
        output = model(data)   # Forward pass

        # Compute loss
        loss = criterion(output, target)  
        loss.backward()        # Backward pass
        optimizer.step()       # Optimize parameters
        
        train_loss += loss.item() * data.size(0)  # Accumulate training loss
        pred = output.argmax(dim=1)  # Get predictions
        correct += pred.eq(target).sum().item()  # Count correct predictions

    # Average training loss and accuracy
    train_loss /= len(train_loader.dataset)
    train_accuracy = correct / len(train_loader.dataset)

    print(f'Epoch: {epoch + 1} \tTraining Loss: {train_loss:.6f} \tTraining Accuracy: {100. * train_accuracy:.2f}%')

    # Validation Phase
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct = 0

    with torch.no_grad():  # No gradient calculation during validation
        for batch_idx, (data, target) in enumerate(val_loader):
            print(f"Validating batch {batch_idx + 1}/{len(val_loader)}")  # Debugging print
            data = data.to(device)  # Move data to the same device
            target = target.to(device)

            output = model(data)  # Forward pass
            loss = criterion(output, target)  # Compute loss
            val_loss += loss.item() * data.size(0)  # Accumulate validation loss
            pred = output.argmax(dim=1)  # Get predictions
            correct += pred.eq(target).sum().item()  # Count correct predictions

    # Average validation loss and accuracy
    val_loss /= len(val_loader.dataset)
    val_accuracy = correct / len(val_loader.dataset)

    print(f'Validation Loss: {val_loss:.6f}, Validation Accuracy: {100. * val_accuracy:.2f}%')


Starting Epoch 1/1
Processing batch 1/94
Processing batch 2/94
Processing batch 3/94
Processing batch 4/94
Processing batch 5/94
Processing batch 6/94
Processing batch 7/94
Processing batch 8/94
Processing batch 9/94
Processing batch 10/94
Processing batch 11/94
Processing batch 12/94
Processing batch 13/94
Processing batch 14/94
Processing batch 15/94
Processing batch 16/94
Processing batch 17/94
Processing batch 18/94
Processing batch 19/94
Processing batch 20/94
Processing batch 21/94
Processing batch 22/94
Processing batch 23/94
Processing batch 24/94
Processing batch 25/94
Processing batch 26/94
Processing batch 27/94
Processing batch 28/94
Processing batch 29/94
Processing batch 30/94
Processing batch 31/94
Processing batch 32/94
Processing batch 33/94
Processing batch 34/94
Processing batch 35/94
Processing batch 36/94
Processing batch 37/94
Processing batch 38/94
Processing batch 39/94
Processing batch 40/94
Processing batch 41/94
Processing batch 42/94
Processing batch 43/94
P

In [28]:
# Save the model
torch.save(model.state_dict(), 'flower_model.pth')